# Install Python Libraries

In [ ]:
import sys
!{sys.executable} -m pip install pyspark
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install requests

# ETL Revegy Data into Pandas df

In [ ]:
import requests, json
import pprint
import re
import pandas as pd
import csv
import os
import sys
import time

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

start = time.time()

data = { 'username' : 'INSERT USERNAME','password' : 'INSERT PASSWORD!'}
# Step 1: input your revegy username & password above

with requests.Session() as s:
    r = s.post('http://rv1.revegy.com/api/auth/web-login',data=data)
    text = r.text.encode("utf-8")
    bearer_token = str(text).split('{"access_token":"')[1].split('","current_user"')[0]

print('bearer_token : %s\n' % bearer_token)

headers = {'Authorization' : 'Bearer %s' % bearer_token}


# ------------------------------------------------------------------------------------------------------------------ #
# Provided below are a the lists and dictionaries that we'll be using to store some metadata for
# ------------------------------------------------------------------------------------------------------------------ #
aggregateObjList = ['658cd436-d604-407f-a4c9-1685da99403a','affc3ff9-9d6f-48b3-bbbe-e216ab2ec82e','b1523c78-abe3-4c89-9393-1d1b0760593e']
# aggregateObjList is a list of Plan Objects (PO) that you don't want to pull a list of fields for
# or pull metadata for. This will vary, and may not be necessary for your use case

accountKeyList =[]
planKeyList    =[]
oppKeyList     =[]

objkeyDict     ={}
revegyDict     ={}
fieldKeyDict   ={}

# ------------------------------------------------------------------------------------------------------------------ #
# what does the below helper function do below?
# getAccountIds() pulls a list of all ACCOUNTS that exist within revegy's database. 
# fields being pulled: ACCOUNT name, revegy ACCOUNT id (not the same as sfdc accountid), and revegy ACCOUNT key

# outside of pulling the ACCOUNT information above, why do we have this helper function?
# the ACCOUNT key information will be referenced in another helper function below - revegyFields()
# we'll be looping through all of your plan objects, and will need to check each key to see if each plan object has
# data for each account key contained in revegy (ie. because plan objects can be tied to opportunities or accounts)
# ------------------------------------------------------------------------------------------------------------------ #
def getAccountIds():
    endpoint='http://api.revegy.com/RevegyServer/api/accounts/all-accounts'
    r = requests.get(endpoint, headers=headers)
    data = r.json()

    Name = [item['name'] for item in data]
    Id   = [item['id']   for item in data]
    Key  = [item['key']  for item in data]
    
    for i in range(0,str(data).count('key')):
        try:
            key  = data[i]['key']
            accountKeyList.append(key)
        except:
            continue
    print('\nTotal Number of Accoount Keys Added : %s\n' % len(accountKeyList))
    return accountKeyList

# ------------------------------------------------------------------------------------------------------------------ #
# what does the below helper function do below?
# getOpportunityIds() pulls a list of all OPPORTUNITIES that exist within revegy's database. 
# fields being pulled: OPPORTUNITY name, revegy OPPORTUNITY id (not the same as sfdc opportunityid), & revegy OPPORTUNITY key

# outside of pulling the OPPORTUNITY information above, why do we have this helper function?
# the OPPORTUNITY key information will be referenced in another helper function below - revegyFields()
# we'll be looping through all of your plan objects, and will need to check each key to see if each plan object has
# data for each OPPORTUNITY key contained in revegy (ie. because plan objects can be tied to opportunities or accounts)
# ------------------------------------------------------------------------------------------------------------------ #
def getOpportunityIds():
    endpoint='http://api.revegy.com/RevegyServer/api/opportunities/all-opportunities'
    r = requests.get(endpoint, headers=headers)
    data = r.json()

    Name = [item['name'] for item in data]
    Id   = [item['id']   for item in data]
    Key  = [item['key']  for item in data]
    
    for i in range(0,str(data).count('key')):
        try:
            key  = data[i]['key']
            oppKeyList.append(key)
        except:
            continue
    print('\nTotal Number of Opportunity Keys Added : %s\n' % len(oppKeyList))
    return oppKeyList

# ------------------------------------------------------------------------------------------------------------------ #
# what does the below helper function do below?
# getPlanKeys() pulls a list of all plan objects that exist within revegy's database. 
# This will provide a data on the 1. Plan object (PO) key, 2. PO name, 3. PO Reuse Level, 4. PO Type

# Outside of pulling the PLAN OBJECT information above, why do we have this helper function?
# the PLAN OBJECT key information will be referenced in another helper function below - revegyFields()
# for each PLAN OBJECT key, we'll be checking to see if there's any account or opportunity data for us to pull
# ------------------------------------------------------------------------------------------------------------------ #
def getPlanKeys(x,y): 
# where 'x' is the api endpoint string & 'y' is the name of the list (no action needed)
    endpoint='http://api.revegy.com/RevegyServer/'+ x
    r = requests.get(endpoint, headers=headers)
    data = r.json()
    try:
        key        = [item['key'] for item in data]
        name       = [item['name'] for item in data]
        reuseLevel = [item['reuseLevel'] for item in data]
        Type       = [item['type'] for item in data]

        df=  pd.DataFrame(
          {'key': key,
           'name': name,
           'reuseLevel': reuseLevel,
           'type': Type
          })
        print(df)
        
    except:
         print('degging error: 1st in getPlanKeys()')

    for i in range(0,len(data)):
        try:
            if x.find('planobject') == -1: 
            # if 'planobject' string is not found in x (where x is the api endpoint)
            # no action needed here. this is just in case the api endpoint were to be changed
                opportunityNumber = data[i]['opportunityNumber']
                key = data[i]['key']
                oppKeyDict[key]=({'opportunityNumber':opportunityNumber})
                y.append(key)
            else:
                name = data[i]['name']
                key = data[i]['key']
                objkeyDict[key]=({'name':name})
                y.append(key)
        except:
            continue
    print('\nTotal Number of %s-keys Added : %s\n' % (str(x).split('/')[1].split('/')[0],len(y)))
    
    if x == 'api/planobject/plan-object-list':
        for q in aggregateObjList:
            y.remove(q)
        return y
    else:
        return y

# ------------------------------------------------------------------------------------------------------------------ #
# what does the below helper function do below?
# revegyFields() pulls a list of all fields that belong to a plan object

# why do we have this helper function?
# this function will allow us to name the each field (ie. column) in the table that eventually gets created
# ------------------------------------------------------------------------------------------------------------------ #
def revegyFields():
    for item in planKeyList:
        endpointList = []
        endpointList.append('http://api.revegy.com/RevegyServer/api/planobject/ACCOUNT/%s/%s/get' % (accountKeyList[0],item))
        endpointList.append('http://api.revegy.com/RevegyServer/api/planobject/OPPORTUNITY/%s/%s/get' % (oppKeyList[0],item))
        for x in endpointList:
            r = requests.get(x, headers=headers)
            data = json.dumps(r.json(), sort_keys=True, indent=4)
            r = r.json()  
            try:
                for i in range(0,len(r['planObjectItemSet'])):
                    planObjectItemSet    = r['planObjectItemSet'][i]
                    objectKey            = r['key']
                    objectName           = r['name']
                    fieldKey             = planObjectItemSet['key']
                    fieldName            = planObjectItemSet['question']
                    isRequired           = planObjectItemSet['required']
                    responseType         = planObjectItemSet['responseType']
                    integrationEnabled   = planObjectItemSet['integrationEnabled']
                    integrationField     = planObjectItemSet['integrationField']
                    integrationDirection = planObjectItemSet['integrationDirection']
                    if len(fieldKey)  == 18:
                        continue
                    else:
                        fieldKeyDict[fieldKey]=(
                            {
                                'planObjectKey':objectName,
                                'fieldName':re.sub('_esponse','_response',fieldName),
                                'integrationEnabled':integrationEnabled,
                                'isRequired':isRequired,
                                'responseType':responseType
                            }   
                        )
            except:
                continue
    print('\nTotal Number of Field Keys Added : %s\n' % len(fieldKeyDict))

# ------------------------------------------------------------------------------------------------------------------ #
# what does the below helper function do below?
# getCleanData() will go through all of your available export ids that are contained within the revegy app
# you can reference a list of the export plan objects that your CSM has configured by going to:
# 1. the revegy app, 2. the import/export section, 3. clicking export.
# a list of the available options should display afterwards

# it will loop through each export id, and then make an api request call that will 'programmatically' export the report
# once exported, it will then parse the report's json, organize the data into a data structure
# the data structure will be stored within the 'revegyDict' dictionary
# we'll be using the 'revegyDict' dictionary to eventually create the underlying data table
# ------------------------------------------------------------------------------------------------------------------ #
def getCleanData(z):
    t0 = time.time() 
    
    if z == 'OPPYPROFILE':
        endpoint = 'http://api.revegy.com/RevegyServer/api/export/%s' % z
    else:
        endpoint = 'http://api.revegy.com/RevegyServer/api/export/PO_%s' % z
    r = requests.post(endpoint, headers=headers)
    data = r.json()
    exportid = data['response']

    time.sleep(12) 
    # give export time to process

    endpoint = 'http://api.revegy.com/RevegyServer/api/export/' + exportid + '/results'
    r = requests.get(endpoint, headers=headers)
    data = r.json()["Records"]

    if len(z) < 36: 
    # if Plan Object (PO) export is not 36 char id (ie. an example of this is the OPPYPROFILE PO)
        for x in range(0,len(data)):
            sfdc_id        = data[x]['crmxref']
            planObjectKey  = z
            planObjectName = z
            fieldKey       = 'None'
            raw_fieldKey = [key for key in data[x] if str(key) != 'crmxref' and str(key) != 'id'  and str(key) != 'currency' \
                            and str(key) != 'name' and str(key) != 'pok' and str(key) != 'externalid' and str(key) != 'seq (row number)' ] # IMPORTANT
            
            if sfdc_id == '' or str(sfdc_id) == 'None':
                continue
            for y in raw_fieldKey:
                if str(y).find(' - ') == -1:
                    fieldName = str(y).rstrip()
                else:
                    fieldName = str(y)[int(str(y).find(' - '))+2:].rstrip()

                fieldName =  ''.join(e for e in fieldName if e.isalnum())
                schema    = str(planObjectName) + '_' + fieldName
                response  = data[x][y]   

                values = (
                    {
                        'planObjectName':planObjectName,
                        'planObjectKey':planObjectKey,
                        'fieldName':re.sub('_esponse','_response',fieldName),
                        'fieldKey':fieldKey,
                        'schema':schema,
                        'response':response
                    }   
                )
                revegyDict.setdefault(sfdc_id,[]).append(values) 

    else:
        for x in range(0,len(data)):
            sfdc_id        = data[x]['crmxref']
            planObjectKey  = data[x]['pok']
            planObjectName = objkeyDict[planObjectKey]['name']
            planObjectName =  ''.join(e for e in planObjectName if e.isalnum())
            raw_fieldKey = [key for key in data[x] if str(key) != 'crmxref' and str(key) != 'id' \
                            and str(key) != 'name' and str(key) != 'pok' and str(key) != 'externalid' and str(key) != 'seq (row number)' ] # IMPORTANT
            
            if sfdc_id == '' or str(sfdc_id) == 'None':
                continue
            for y in raw_fieldKey:
                fieldKey  = str(y)[0:int(str(y).find(' '))].rstrip()
                fieldName = str(y)[int(str(y).find(' '))+2:].rstrip()

                try:
                    if fieldKeyDict[fieldKey[len(fieldKey)-36:len(fieldKey)]]['responseType'] == 11: # if it's a color indicator field
                        fieldName = 'colorcode'+str(y)[int(str(y).find(' '))+2:].rstrip()
                    else:
                        fieldName = str(y)[int(str(y).find(' '))+2:].rstrip()
                except:
                    fieldName = str(y)[int(str(y).find(' '))+2:].rstrip()

                if fieldName[len(fieldName)-1:-1] == ' ':
                    fieldName = fieldName[:-1]
                else:
                    fieldName = fieldName
                
                fieldName =  ''.join(e for e in fieldName if e.isalnum())
                schema    = planObjectName + '_' + fieldName
                response  = data[x][y]

                values = (
                    {
                        'planObjectName':planObjectName,
                        'planObjectKey':planObjectKey,
                        'fieldName':re.sub('_esponse','_response',fieldName),
                        'fieldKey':fieldKey,
                        'schema':schema,
                        'response':response
                    }   
                )
                revegyDict.setdefault(sfdc_id,[]).append(values)

    t1 = time.time()
    total = t1-t0
    print('Debug : finished printing data for PO in %s seconds' % (round(total,2)))
    
    return revegyDict

# ------------------------------------------------------------------------------------------------------------------ #
# call helper functions in correct order
# ------------------------------------------------------------------------------------------------------------------ #
getAccountIds()
getOpportunityIds()
getPlanKeys('api/planobject/plan-object-list',planKeyList)
revegyFields()

planKeyList = ['OPPYPROFILE'] + planKeyList
# NEEDS TO BE REVIEWED
# the line above may/may not need to be removed depending on how your export ids have been configured
# if your revegy CSM has configured the OPPYPROFILE export id, then you can keep the line above
# if the OPPYPROFILE export id has not been configured, then the above line can be removed

for item in planKeyList[:3]: # Added for testing purposes --> eventually can be removed
    getCleanData(item)

df = pd.DataFrame([z for z in revegyDict], columns =['sfdc_id']) 
for item in revegyDict:
  for i in range(0,3):
                 #len(revegyDict[item])):
                 #the above line is blocked off for testing purposes
    try:
        if {revegyDict[item][i]['schema']}.issubset(df.columns): 
            # check to see if column has already been created. If so, update the already existing column's cell
            df.at[df[df['sfdc_id']==item].index.values,revegyDict[item][i]['schema']] = revegyDict[item][i]['response']
        else:
            try:
                df[revegyDict[item][i]['schema']]= df['sfdc_id'].apply(lambda x,axis=1: revegyDict[item][i]['response'] if revegyDict[x]==revegyDict[item] else '')
            except:
                #print('debugging error: 1st')
                continue
    except:
        #print('debugging error: 2nd')
        continue

end = time.time()
total = end-start
print('\nTime it took script to run: %s minute(s) or %s hour(s)' % (round(total/60,2),round(total/60/60,2)))

In [ ]:
display(df)